# Tier 3: Time Series Analysis & Forecasting

---

**Author:** Brandon Deloatch
**Affiliation:** Quipu Research Labs, LLC
**Date:** 2025-10-02
**Version:** v1.3
**License:** MIT
**Notebook ID:** d285be7b-652a-451c-8633-4a3473e9272a

---

## Citation
Brandon Deloatch, "Tier 3: Time Series Analysis & Forecasting," Quipu Research Labs, LLC, v1.3, 2025-10-02.

Please cite this notebook if used or adapted in publications, presentations, or derivative work.

---

## Contributors / Acknowledgments
- **Primary Author:** Brandon Deloatch (Quipu Research Labs, LLC)
- **Institutional Support:** Quipu Research Labs, LLC - Advanced Analytics Division
- **Technical Framework:** Built on scikit-learn, pandas, numpy, and plotly ecosystems
- **Methodological Foundation:** Statistical learning principles and modern data science best practices

---

## Version History
| Version | Date | Notes |
|---------|------|-------|
| v1.3 | 2025-10-02 | Enhanced professional formatting, comprehensive documentation, interactive visualizations |
| v1.2 | 2024-09-15 | Updated analysis methods, improved data generation algorithms |
| v1.0 | 2024-06-10 | Initial release with core analytical framework |

---

## Environment Dependencies
- **Python:** 3.8+
- **Core Libraries:** pandas 2.0+, numpy 1.24+, scikit-learn 1.3+
- **Visualization:** plotly 5.0+, matplotlib 3.7+
- **Statistical:** scipy 1.10+, statsmodels 0.14+
- **Development:** jupyter-lab 4.0+, ipywidgets 8.0+

> **Reproducibility Note:** Use requirements.txt or environment.yml for exact dependency matching.

---

## Data Provenance
| Dataset | Source | License | Notes |
|---------|--------|---------|-------|
| Synthetic Data | Generated in-notebook | MIT | Custom algorithms for realistic simulation |
| Statistical Distributions | NumPy/SciPy | BSD-3-Clause | Standard library implementations |
| ML Algorithms | Scikit-learn | BSD-3-Clause | Industry-standard implementations |
| Visualization Schemas | Plotly | MIT | Interactive dashboard frameworks |

---

## Execution Provenance Logs
- **Created:** 2025-10-02
- **Notebook ID:** d285be7b-652a-451c-8633-4a3473e9272a
- **Execution Environment:** Jupyter Lab / VS Code
- **Computational Requirements:** Standard laptop/workstation (2GB+ RAM recommended)

> **Auto-tracking:** Execution metadata can be programmatically captured for reproducibility.

---

## Disclaimer & Responsible Use
This notebook is provided "as-is" for educational, research, and professional development purposes. Users assume full responsibility for any results, applications, or decisions derived from this analysis.

**Professional Standards:**
- Validate all results against domain expertise and additional data sources
- Respect licensing and attribution requirements for all dependencies
- Follow ethical guidelines for data analysis and algorithmic decision-making
- Credit all methodological sources and derivative frameworks appropriately

**Academic & Commercial Use:**
- Permitted under MIT license with proper attribution
- Suitable for educational curriculum and professional training
- Appropriate for commercial adaptation with citation requirements
- Recommended for reproducible research and transparent analytics

---



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

print(" Tier 3: Time Series Analysis & Forecasting")
print("=" * 47)
print(" CROSS-REFERENCES:")
print("• Prerequisites: Tier1_Descriptive.ipynb, Tier1_Pivot.ipynb")
print("• Builds On: Tier3_TimeSeriesDecomposition.ipynb, Tier3_ARIMA.ipynb")
print("• Complements: Tier3_ExponentialSmoothing.ipynb, Tier3_MovingAverages.ipynb")
print("• Advanced: Tier3_FourierAnalysis.ipynb, Tier3_WaveletAnalysis.ipynb")
print("=" * 47)
print("Time Series Techniques:")
print("• Seasonal decomposition and trend analysis")
print("• Multiple forecasting methods comparison")
print("• Stationarity testing and differencing")
print("• Autocorrelation and partial autocorrelation analysis")
print("• Forecast accuracy evaluation and confidence intervals")

 Tier 3: Time Series Analysis & Forecasting
 CROSS-REFERENCES:
• Prerequisites: Tier1_Descriptive.ipynb, Tier1_Pivot.ipynb
• Builds On: Tier3_TimeSeriesDecomposition.ipynb, Tier3_ARIMA.ipynb
• Complements: Tier3_ExponentialSmoothing.ipynb, Tier3_MovingAverages.ipynb
• Advanced: Tier3_FourierAnalysis.ipynb, Tier3_WaveletAnalysis.ipynb
Time Series Techniques:
• Seasonal decomposition and trend analysis
• Multiple forecasting methods comparison
• Stationarity testing and differencing
• Autocorrelation and partial autocorrelation analysis
• Forecast accuracy evaluation and confidence intervals


In [3]:
# Generate comprehensive synthetic time series datasets
np.random.seed(42)

def generate_time_series_datasets():
    """Generate multiple realistic time series for analysis."""

    # Dataset 1: Monthly Sales Data (with trend and seasonality)
    dates_monthly = pd.date_range('2020-01-01', periods=48, freq='M')
    n_months = len(dates_monthly)

    # Base trend component
    trend = np.linspace(100000, 150000, n_months)

    # Seasonal component (stronger in Q4, weaker in Q1)
    seasonal = 10000 * np.sin(2 * np.pi * np.arange(n_months) / 12) + \
               5000 * np.sin(4 * np.pi * np.arange(n_months) / 12)

    # Add holiday spikes and random noise
    holiday_spikes = np.zeros(n_months)
    holiday_months = [11, 23, 35, 47] # December months
    for month in holiday_months:
        if month < n_months:
            holiday_spikes[month] = np.random.normal(15000, 3000)

    noise = np.random.normal(0, 5000, n_months)

    sales_data = trend + seasonal + holiday_spikes + noise
    sales_data = np.maximum(sales_data, 50000) # Minimum sales floor

    # Dataset 2: Daily Website Traffic (with weekly patterns)
    dates_daily = pd.date_range('2023-01-01', periods=365, freq='D')
    n_days = len(dates_daily)

    # Weekly seasonality (lower on weekends)
    day_of_week = np.array([d.weekday() for d in dates_daily])
    weekly_pattern = np.where(day_of_week < 5, 1.0, 0.6) # Weekdays vs weekends

    # Base traffic with growth trend
    base_traffic = 10000 + np.linspace(0, 5000, n_days)

    # Seasonal component (higher in winter months)
    yearly_seasonal = 2000 * np.sin(2 * np.pi * np.arange(n_days) / 365.25 + np.pi)

    # Random events and noise
    random_events = np.random.poisson(0.05, n_days) * np.random.normal(5000, 1000, n_days)
    daily_noise = np.random.normal(0, 1000, n_days)

    traffic_data = base_traffic * weekly_pattern + yearly_seasonal + random_events + daily_noise
    traffic_data = np.maximum(traffic_data, 1000)

    # Dataset 3: Hourly Temperature Data (with daily and seasonal cycles)
    dates_hourly = pd.date_range('2023-01-01', periods=24*90, freq='H') # 90 days
    n_hours = len(dates_hourly)

    # Daily temperature cycle
    hour_of_day = np.array([d.hour for d in dates_hourly])
    daily_cycle = 10 * np.sin(2 * np.pi * (hour_of_day - 6) / 24) # Peak at 2 PM

    # Seasonal trend (winter to spring)
    day_of_year = np.array([d.dayofyear for d in dates_hourly])
    seasonal_trend = 15 * np.sin(2 * np.pi * day_of_year / 365.25 - np.pi/2) + 20

    # Weather noise
    weather_noise = np.random.normal(0, 3, n_hours)

    temperature_data = seasonal_trend + daily_cycle + weather_noise

    # Create DataFrames
    sales_df = pd.DataFrame({
        'date': dates_monthly,
        'sales': sales_data,
        'month': dates_monthly.month,
        'quarter': dates_monthly.quarter,
        'year': dates_monthly.year
    })

    traffic_df = pd.DataFrame({
        'date': dates_daily,
        'traffic': traffic_data,
        'day_of_week': day_of_week,
        'is_weekend': day_of_week >= 5,
        'month': dates_daily.month
    })

    temperature_df = pd.DataFrame({
        'datetime': dates_hourly,
        'temperature': temperature_data,
        'hour': hour_of_day,
        'day': dates_hourly.day
    })

    return sales_df, traffic_df, temperature_df

# Generate datasets
sales_df, traffic_df, temperature_df = generate_time_series_datasets()

print(" Time Series Datasets Created:")
print(f"\n1. Monthly Sales Data:")
print(f" • Period: {sales_df['date'].min()} to {sales_df['date'].max()}")
print(f" • Data points: {len(sales_df)}")
print(f" • Sales range: ${sales_df['sales'].min():,.0f} - ${sales_df['sales'].max():,.0f}")

print(f"\n2. Daily Website Traffic:")
print(f" • Period: {traffic_df['date'].min()} to {traffic_df['date'].max()}")
print(f" • Data points: {len(traffic_df)}")
print(f" • Traffic range: {traffic_df['traffic'].min():,.0f} - {traffic_df['traffic'].max():,.0f} visits")

print(f"\n3. Hourly Temperature Data:")
print(f" • Period: {temperature_df['datetime'].min()} to {temperature_df['datetime'].max()}")
print(f" • Data points: {len(temperature_df)}")
print(f" • Temperature range: {temperature_df['temperature'].min():.1f}°C - {temperature_df['temperature'].max():.1f}°C")

# Display sample data
print(f"\nSample Sales Data:")
print(sales_df.head())

 Time Series Datasets Created:

1. Monthly Sales Data:
 • Period: 2020-01-31 00:00:00 to 2023-12-31 00:00:00
 • Data points: 48
 • Sales range: $89,023 - $158,314

2. Daily Website Traffic:
 • Period: 2023-01-01 00:00:00 to 2023-12-31 00:00:00
 • Data points: 365
 • Traffic range: 3,034 - 22,798 visits

3. Hourly Temperature Data:
 • Period: 2023-01-01 00:00:00 to 2023-03-31 23:00:00
 • Data points: 2160
 • Temperature range: -13.3°C - 33.1°C

Sample Sales Data:
        date          sales  month  quarter  year
0 2020-01-31   98829.233126      1        1  2020
1 2020-02-29  109223.272021      2        1  2020
2 2020-03-31  123014.104709      3        1  2020
3 2020-04-30  117028.663007      4        2  2020
4 2020-05-31  106238.074238      5        2  2020


In [5]:
# 1. TIME SERIES DECOMPOSITION AND ANALYSIS
print(" 1. TIME SERIES DECOMPOSITION AND ANALYSIS")
print("=" * 45)

# Decompose the monthly sales time series
sales_ts = sales_df.set_index('date')['sales']

# Perform seasonal decomposition
decomposition = seasonal_decompose(sales_ts, model='additive', period=12)

print("Sales Time Series Decomposition:")
print(f"• Original series: {len(sales_ts)} monthly observations")
print(f"• Decomposition model: Additive")
print(f"• Seasonal period: 12 months")

# Calculate component statistics
trend_component = decomposition.trend.dropna()
seasonal_component = decomposition.seasonal
residual_component = decomposition.resid.dropna()

print(f"\nComponent Analysis:")
print(f"• Trend variation: {trend_component.std():.0f} (${trend_component.min():,.0f} to ${trend_component.max():,.0f})")
print(f"• Seasonal amplitude: ±{seasonal_component.abs().max():.0f}")
print(f"• Residual std: {residual_component.std():.0f}")

# Stationarity test
def test_stationarity(timeseries, title="Time Series"):
    """Perform Augmented Dickey-Fuller test for stationarity."""
    result = adfuller(timeseries.dropna())

    print(f"\n{title} - Stationarity Test (Augmented Dickey-Fuller):")
    print(f"• ADF Statistic: {result[0]:.6f}")
    print(f"• p-value: {result[1]:.6f}")
    print(f"• Critical Values:")
    for key, value in result[4].items():
        print(f" - {key}: {value:.3f}")

    if result[1] <= 0.05:
        print(" Series is stationary (reject null hypothesis)")
    else:
        print(" Series is non-stationary (fail to reject null hypothesis)")

    return result[1] <= 0.05

# Test stationarity of original and differenced series
is_stationary_original = test_stationarity(sales_ts, "Original Sales Series")

# First difference
sales_diff = sales_ts.diff().dropna()
is_stationary_diff = test_stationarity(sales_diff, "First Differenced Series")

# Autocorrelation analysis
sales_autocorr = acf(sales_ts.dropna(), nlags=24, fft=False)
sales_partial_autocorr = pacf(sales_ts.dropna(), nlags=24)

print(f"\nAutocorrelation Analysis:")
print(f"• Lag 1 autocorrelation: {sales_autocorr[1]:.3f}")
print(f"• Lag 12 autocorrelation: {sales_autocorr[12]:.3f}")
print(f"• Significant lags (>0.5): {np.sum(np.abs(sales_autocorr) > 0.5)}")

 1. TIME SERIES DECOMPOSITION AND ANALYSIS
Sales Time Series Decomposition:
• Original series: 48 monthly observations
• Decomposition model: Additive
• Seasonal period: 12 months

Component Analysis:
• Trend variation: 11557 ($106,372 to $144,181)
• Seasonal amplitude: ±19162
• Residual std: 3474

Original Sales Series - Stationarity Test (Augmented Dickey-Fuller):
• ADF Statistic: -0.466426
• p-value: 0.898403
• Critical Values:
 - 1%: -3.621
 - 5%: -2.944
 - 10%: -2.610
 Series is non-stationary (fail to reject null hypothesis)

First Differenced Series - Stationarity Test (Augmented Dickey-Fuller):
• ADF Statistic: -8.361074
• p-value: 0.000000
• Critical Values:
 - 1%: -3.627
 - 5%: -2.946
 - 10%: -2.612
 Series is stationary (reject null hypothesis)

Autocorrelation Analysis:
• Lag 1 autocorrelation: 0.722
• Lag 12 autocorrelation: 0.333
• Significant lags (>0.5): 4


In [6]:
# 2. MULTIPLE FORECASTING METHODS COMPARISON
print(" 2. MULTIPLE FORECASTING METHODS COMPARISON")
print("=" * 47)

# Split data for train/test
train_size = int(len(sales_ts) * 0.8)
train_data = sales_ts[:train_size]
test_data = sales_ts[train_size:]

print(f"Data Split:")
print(f"• Training period: {train_data.index[0]} to {train_data.index[-1]} ({len(train_data)} months)")
print(f"• Testing period: {test_data.index[0]} to {test_data.index[-1]} ({len(test_data)} months)")

# Method 1: Simple Moving Average
def moving_average_forecast(data, window=3, horizon=None):
    """Simple moving average forecast."""
    if horizon is None:
        horizon = len(test_data)

    forecasts = []
    last_values = data.tail(window).values

    for _ in range(horizon):
        forecast = np.mean(last_values)
        forecasts.append(forecast)
        # Update rolling window
        last_values = np.append(last_values[1:], forecast)

    return np.array(forecasts)
# Method 2: Exponential Smoothing (Holt-Winters)
exp_smoothing_model = ExponentialSmoothing(
    train_data,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit()
exp_smoothing_forecast = exp_smoothing_model.forecast(len(test_data))

# Method 3: ARIMA Model
# Find optimal ARIMA parameters (simplified grid search)
def find_best_arima(data, max_p=3, max_d=2, max_q=3):
    """Find best ARIMA parameters using AIC."""
    best_aic = np.inf
    best_params = None

    for p in range(max_p + 1):
        for d in range(max_d + 1):
            for q in range(max_q + 1):
                try:
                    model = ARIMA(data, order=(p, d, q))
                    fitted_model = model.fit()
                    if fitted_model.aic < best_aic:
                        best_aic = fitted_model.aic
                        best_params = (p, d, q)
                except:
                    continue

    return best_params, best_aic
# Find best ARIMA parameters
best_arima_params, best_aic = find_best_arima(train_data)
print(f"\nBest ARIMA parameters: {best_arima_params} (AIC: {best_aic:.2f})")

# Fit best ARIMA model
arima_model = ARIMA(train_data, order=best_arima_params).fit()
arima_forecast = arima_model.forecast(len(test_data))

# Method 4: Naive forecasts (baseline)
naive_forecast = np.full(len(test_data), train_data.iloc[-1])
seasonal_naive_forecast = np.tile(train_data.iloc[-12:].values, (len(test_data) // 12 + 1))[:len(test_data)]

# Generate forecasts
ma_forecast = moving_average_forecast(train_data, window=3)

# Compile all forecasts
forecasts = {
    'Moving Average (3)': ma_forecast,
    'Exponential Smoothing': exp_smoothing_forecast.values,
    'ARIMA': arima_forecast.values,
    'Naive': naive_forecast,
    'Seasonal Naive': seasonal_naive_forecast
}
# Calculate accuracy metrics
def calculate_accuracy_metrics(actual, predicted):
    """Calculate comprehensive accuracy metrics."""
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape
    }
# Evaluate all methods
accuracy_results = {}
for method_name, forecast_values in forecasts.items():
    metrics = calculate_accuracy_metrics(test_data.values, forecast_values)
    accuracy_results[method_name] = metrics

    print(f"\n{method_name}:")
    print(f" • MAE: {metrics['MAE']:,.0f}")
    print(f" • RMSE: {metrics['RMSE']:,.0f}")
    print(f" • MAPE: {metrics['MAPE']:.1f}%")
# Find best performing method
best_method = min(accuracy_results.keys(), key=lambda x: accuracy_results[x]['MAPE'])
best_mape = accuracy_results[best_method]['MAPE']
print(f"\n Best Performing Method: {best_method} (MAPE: {best_mape:.1f}%)")

 2. MULTIPLE FORECASTING METHODS COMPARISON
Data Split:
• Training period: 2020-01-31 00:00:00 to 2023-02-28 00:00:00 (38 months)
• Testing period: 2023-03-31 00:00:00 to 2023-12-31 00:00:00 (10 months)

Best ARIMA parameters: (0, 1, 0) (AIC: 798.82)

Moving Average (3):
 • MAE: 7,240
 • RMSE: 8,434
 • MAPE: 5.1%

Exponential Smoothing:
 • MAE: 4,237
 • RMSE: 5,363
 • MAPE: 3.0%

ARIMA:
 • MAE: 7,722
 • RMSE: 9,548
 • MAPE: 5.5%

Naive:
 • MAE: 7,722
 • RMSE: 9,548
 • MAPE: 5.5%

Seasonal Naive:
 • MAE: 12,830
 • RMSE: 14,370
 • MAPE: 8.9%

 Best Performing Method: Exponential Smoothing (MAPE: 3.0%)

Best ARIMA parameters: (0, 1, 0) (AIC: 798.82)

Moving Average (3):
 • MAE: 7,240
 • RMSE: 8,434
 • MAPE: 5.1%

Exponential Smoothing:
 • MAE: 4,237
 • RMSE: 5,363
 • MAPE: 3.0%

ARIMA:
 • MAE: 7,722
 • RMSE: 9,548
 • MAPE: 5.5%

Naive:
 • MAE: 7,722
 • RMSE: 9,548
 • MAPE: 5.5%

Seasonal Naive:
 • MAE: 12,830
 • RMSE: 14,370
 • MAPE: 8.9%

 Best Performing Method: Exponential Smoothing (M

In [7]:
# 3. INTERACTIVE TIME SERIES VISUALIZATIONS
print(" 3. INTERACTIVE TIME SERIES VISUALIZATIONS")
print("=" * 47)

# Create comprehensive time series dashboard
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=[
        'Time Series Decomposition (Sales Data)',
        'Autocorrelation and Partial Autocorrelation',
        'Forecast Comparison (Multiple Methods)',
        'Daily Traffic Patterns',
        'Hourly Temperature Cycles',
        'Forecast Accuracy Comparison'
    ],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Time Series Decomposition
fig.add_trace(
    go.Scatter(
        x=sales_ts.index,
        y=sales_ts.values,
        mode='lines',
        name='Original',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=trend_component.index,
        y=trend_component.values,
        mode='lines',
        name='Trend',
        line=dict(color='red', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=seasonal_component.index,
        y=seasonal_component.values,
        mode='lines',
        name='Seasonal',
        line=dict(color='green', width=2)
    ),
    row=1, col=1
)

# 2. Autocorrelation plots
lags = np.arange(len(sales_autocorr))
fig.add_trace(
    go.Bar(
        x=lags,
        y=sales_autocorr,
        name='ACF',
        marker_color='lightblue'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        x=lags,
        y=sales_partial_autocorr,
        name='PACF',
        marker_color='lightcoral'
    ),
    row=1, col=2
)

# 3. Forecast Comparison
# Plot historical data
fig.add_trace(
    go.Scatter(
        x=train_data.index,
        y=train_data.values,
        mode='lines',
        name='Training Data',
        line=dict(color='black', width=2)
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=test_data.index,
        y=test_data.values,
        mode='lines',
        name='Actual Test',
        line=dict(color='blue', width=3)
    ),
    row=2, col=1
)

# Plot forecasts
colors = ['red', 'green', 'orange', 'purple', 'brown']
for i, (method, forecast_vals) in enumerate(forecasts.items()):
    fig.add_trace(
        go.Scatter(
            x=test_data.index,
            y=forecast_vals,
            mode='lines',
            name=f'{method}',
            line=dict(color=colors[i % len(colors)], width=2, dash='dash')
        ),
        row=2, col=1
    )

# 4. Daily Traffic Patterns
# Aggregate by day of week
traffic_by_day = traffic_df.groupby('day_of_week')['traffic'].mean()
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig.add_trace(
    go.Bar(
        x=day_names,
        y=traffic_by_day.values,
        name='Avg Traffic by Day',
        marker_color='lightgreen'
    ),
    row=2, col=2
)

# 5. Hourly Temperature Cycles
# Sample a few days for clarity
sample_days = temperature_df[temperature_df['datetime'].dt.day <= 5]

fig.add_trace(
    go.Scatter(
        x=sample_days['datetime'],
        y=sample_days['temperature'],
        mode='lines+markers',
        name='Temperature',
        line=dict(color='red', width=2),
        marker=dict(size=4)
    ),
    row=3, col=1
)

# 6. Forecast Accuracy Comparison
methods = list(accuracy_results.keys())
mape_values = [accuracy_results[method]['MAPE'] for method in methods]
rmse_values = [accuracy_results[method]['RMSE'] for method in methods]

fig.add_trace(
    go.Bar(
        x=methods,
        y=mape_values,
        name='MAPE (%)',
        marker_color='lightcoral',
        yaxis='y6'
    ),
    row=3, col=2
)

# Update layout
fig.update_layout(
    height=1200,
    title="Time Series Analysis & Forecasting Dashboard",
    showlegend=True
)

# Update axis labels
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Lag", row=1, col=2)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Day of Week", row=2, col=2)
fig.update_xaxes(title_text="DateTime", row=3, col=1)
fig.update_xaxes(title_text="Forecast Method", row=3, col=2)

fig.update_yaxes(title_text="Sales ($)", row=1, col=1)
fig.update_yaxes(title_text="Correlation", row=1, col=2)
fig.update_yaxes(title_text="Sales ($)", row=2, col=1)
fig.update_yaxes(title_text="Average Traffic", row=2, col=2)
fig.update_yaxes(title_text="Temperature (°C)", row=3, col=1)
fig.update_yaxes(title_text="MAPE (%)", row=3, col=2)

fig.show()

# Business insights and applications
print(f"\n TIME SERIES BUSINESS INSIGHTS:")
print("=" * 35)

# Sales forecasting insights
current_sales = train_data.iloc[-1]
best_forecast_next = forecasts[best_method][0]
forecast_growth = (best_forecast_next - current_sales) / current_sales

print(f"\n1. Sales Forecasting Analysis:")
print(f" • Current monthly sales: ${current_sales:,.0f}")
print(f" • Next month forecast ({best_method}): ${best_forecast_next:,.0f}")
print(f" • Projected growth: {forecast_growth*100:+.1f}%")
print(f" • Seasonal peak month: {sales_df.groupby('month')['sales'].mean().idxmax()}")
print(f" • Seasonal low month: {sales_df.groupby('month')['sales'].mean().idxmin()}")

# Annual projections
annual_forecast = forecasts[best_method]
projected_annual_sales = np.sum(annual_forecast) * (12 / len(annual_forecast))
current_annual_run_rate = current_sales * 12

print(f" • Current annual run rate: ${current_annual_run_rate:,.0f}")
print(f" • Projected annual sales: ${projected_annual_sales:,.0f}")

# Traffic patterns insights
weekend_traffic = traffic_df[traffic_df['is_weekend']]['traffic'].mean()
weekday_traffic = traffic_df[~traffic_df['is_weekend']]['traffic'].mean()
traffic_weekend_ratio = weekend_traffic / weekday_traffic

print(f"\n2. Website Traffic Patterns:")
print(f" • Average weekday traffic: {weekday_traffic:,.0f} visits")
print(f" • Average weekend traffic: {weekend_traffic:,.0f} visits")
print(f" • Weekend/Weekday ratio: {traffic_weekend_ratio:.2f}")
print(f" • Peak traffic day: {day_names[traffic_by_day.idxmax()]}")
print(f" • Lowest traffic day: {day_names[traffic_by_day.idxmin()]}")

# ROI of forecasting accuracy
forecast_improvement = (accuracy_results['Naive']['MAPE'] - accuracy_results[best_method]['MAPE']) / accuracy_results['Naive']['MAPE']
inventory_cost_savings = projected_annual_sales * 0.25 * 0.05 * forecast_improvement # 25% inventory cost, 5% improvement
planning_efficiency_gain = 150_000 * forecast_improvement # Annual planning cost improvement

print(f"\n FORECASTING ROI ANALYSIS:")
print("=" * 27)
print(f"• Forecast accuracy improvement: {forecast_improvement*100:.1f}%")
print(f"• Inventory cost savings: ${inventory_cost_savings:,.0f}")
print(f"• Planning efficiency gain: ${planning_efficiency_gain:,.0f}")
print(f"• Total annual benefits: ${inventory_cost_savings + planning_efficiency_gain:,.0f}")

implementation_cost = 125_000
annual_maintenance = 25_000
total_benefits = inventory_cost_savings + planning_efficiency_gain
net_benefits = total_benefits - annual_maintenance
roi = (net_benefits - implementation_cost) / implementation_cost

print(f"• Implementation cost: ${implementation_cost:,.0f}")
print(f"• Annual maintenance: ${annual_maintenance:,.0f}")
print(f"• Net annual benefits: ${net_benefits:,.0f}")
print(f"• ROI: {roi*100:.0f}%")

print(f"\n Cross-Reference Learning Path:")
print(f"• Foundation: Tier3_TimeSeriesDecomposition.ipynb (seasonal analysis)")
print(f"• Advanced: Tier3_ARIMA.ipynb (autoregressive modeling)")
print(f"• Comparison: Tier3_ExponentialSmoothing.ipynb (smoothing methods)")
print(f"• Specialized: Tier3_FourierAnalysis.ipynb (frequency domain)")
print(f"• Complete Guide: CROSS_REFERENCE_GUIDE.md")

 3. INTERACTIVE TIME SERIES VISUALIZATIONS



 TIME SERIES BUSINESS INSIGHTS:

1. Sales Forecasting Analysis:
 • Current monthly sales: $149,549
 • Next month forecast (Exponential Smoothing): $154,015
 • Projected growth: +3.0%
 • Seasonal peak month: 12
 • Seasonal low month: 10
 • Current annual run rate: $1,794,584
 • Projected annual sales: $1,730,270

2. Website Traffic Patterns:
 • Average weekday traffic: 12,734 visits
 • Average weekend traffic: 7,899 visits
 • Weekend/Weekday ratio: 0.62
 • Peak traffic day: Mon
 • Lowest traffic day: Sat

 FORECASTING ROI ANALYSIS:
• Forecast accuracy improvement: 46.6%
• Inventory cost savings: $10,086
• Planning efficiency gain: $69,951
• Total annual benefits: $80,037
• Implementation cost: $125,000
• Annual maintenance: $25,000
• Net annual benefits: $55,037
• ROI: -56%

 Cross-Reference Learning Path:
• Foundation: Tier3_TimeSeriesDecomposition.ipynb (seasonal analysis)
• Advanced: Tier3_ARIMA.ipynb (autoregressive modeling)
• Comparison: Tier3_ExponentialSmoothing.ipynb (smoothin